<a href="https://colab.research.google.com/github/netra-poonia/simulation1/blob/main/Relationship_Manager_Bonus_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## `Base Code`

In [41]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from typing import Dict, List, Tuple, Any
import warnings
warnings.filterwarnings('ignore')

class RMBonusCalculator:
    """
    Complete RM (Relationship Manager) Bonus Calculation System

    Features:
    1. Volume Payout (Slab-based like income tax)
    2. Effort Payout (Matrix-based on disbursals vs logins)
    3. Vintage-based structures (0-3M, 4-6M, 6M+)
    4. Milestone rewards (Silver/Gold/Platinum)
    5. Month evaluation logic based on joining date
    6. Dynamic evaluation period based on financial quarters
    """

    def __init__(self):
        self.setup_configuration()

    def setup_configuration(self):
        """Initialize all configuration data"""

        # Volume payout slabs for different vintage categories
        self.VOLUME_PAYOUT_SLABS = {
            'secured': [
                {'threshold_multiplier': 0, 'bonus_rate': 0.000},
                {'threshold_multiplier': 30, 'bonus_rate': 0.0020},
                {'threshold_multiplier': 50, 'bonus_rate': 0.0025},
                {'threshold_multiplier': 80, 'bonus_rate': 0.004},
                {'threshold_multiplier': 120, 'bonus_rate': 0.005},
            ],
            'secured_direct': [
                {'threshold_multiplier': 0, 'bonus_rate': 0.0012},
                {'threshold_multiplier': 10, 'bonus_rate': 0.0018},
                {'threshold_multiplier': 30, 'bonus_rate': 0.0025},
                {'threshold_multiplier': 60, 'bonus_rate': 0.0030},
            ],
            'unsecured': [
                {'threshold_multiplier': 0, 'bonus_rate': 0.000},
                {'threshold_multiplier': 5, 'bonus_rate': 0.004},
                {'threshold_multiplier': 15, 'bonus_rate': 0.007},
                {'threshold_multiplier': 30, 'bonus_rate': 0.01},
            ],
        }

        # BAU Effort Matrices (logins vs disbursals) - CORRECTED AXES
        self.BAU_SECURED_EFFORT_MATRIX = {
            (0, 0): {(0, 0): 1000, (1, 4): 1200, (5, 6): 1500, (7, float('inf')): 1800},
            (1, 1): {(0, 0): 1200, (1, 4): 1500, (5, 6): 1800, (7, float('inf')): 2000},
            (2, 2): {(0, 0): 1500, (1, 4): 1800, (5, 6): 3000, (7, float('inf')): 3500},
            (3, 3): {(0, 0): 1800, (1, 4): 2000, (5, 6): 3500, (7, float('inf')): 4000},
            (4, float('inf')): {(0, 0): 2000, (1, 4): 3000, (5, 6): 4000, (7, float('inf')): 5000},
        }

        self.BAU_UNSECURED_EFFORT_MATRIX = {
            (0, 0): {(0, 0): 0, (1, 9): 300, (10, 13): 400, (14, float('inf')): 500},
            (1, 3): {(0, 0): 300, (1, 9): 400, (10, 13): 500, (14, float('inf')): 750},
            (4, 4): {(0, 0): 400, (1, 9): 500, (10, 13): 750, (14, float('inf')): 850},
            (5, 7): {(0, 0): 500, (1, 9): 750, (10, 13): 850, (14, float('inf')): 1000},
            (8, float('inf')): {(0, 0): 750, (1, 9): 850, (10, 13): 1000, (14, float('inf')): 1200},
        }

        # Vintage 0-3 Months
        self.VINTAGE_VOLUME_SLABS_0_3M = {
            'secured': [
                {'threshold_multiplier': 0, 'bonus_rate': 0.000},
                {'threshold_multiplier': 5, 'bonus_rate': 0.0025},
                {'threshold_multiplier': 20, 'bonus_rate': 0.0035},
                {'threshold_multiplier': 30, 'bonus_rate': 0.0045},
                {'threshold_multiplier': 45, 'bonus_rate': 0.005},
            ],
            'secured_direct': [
                {'threshold_multiplier': 0, 'bonus_rate': 0.0012},
                {'threshold_multiplier': 10, 'bonus_rate': 0.0016},
                {'threshold_multiplier': 25, 'bonus_rate': 0.0025},
                {'threshold_multiplier': 40, 'bonus_rate': 0.0030},
            ],
            'unsecured': [
                {'threshold_multiplier': 0, 'bonus_rate': 0.000},
                {'threshold_multiplier': 3, 'bonus_rate': 0.004},
                {'threshold_multiplier': 6, 'bonus_rate': 0.008},
                {'threshold_multiplier': 9, 'bonus_rate': 0.01},
            ],
        }

        self.VINTAGE_SECURED_EFFORT_MATRIX_0_3M = {
            (0, 0): {(0, 0): 1500, (1, 1): 1800, (2, float('inf')): 2000},
            (1, 1): {(0, 0): 1800, (1, 1): 2000, (2, float('inf')): 2500},
            (2, 2): {(0, 0): 2500, (1, 1): 3000, (2, float('inf')): 4000},
            (3, float('inf')): {(0, 0): 3000, (1, 1): 3500, (2, float('inf')): 4000},
        }

        self.VINTAGE_UNSECURED_EFFORT_MATRIX_0_3M = {
            (0, 1): {(0, 0): 0, (1, 1): 200, (2, 2): 300, (3, 3): 400, (4, float('inf')): 650},
            (2, 3): {(0, 0): 200, (1, 1): 300, (2, 2): 400, (3, 3): 650, (4, float('inf')): 750},
            (4, 5): {(0, 0): 300, (1, 1): 400, (2, 2): 650, (3, 3): 750, (4, float('inf')): 900},
            (6, float('inf')): {(0, 0): 400, (1, 1): 650, (2, 2): 750, (3, 3): 900, (4, float('inf')): 1200},
        }

        # Vintage 4-6 Months
        self.VINTAGE_VOLUME_SLABS_4_6M = {
            'secured': [
                {'threshold_multiplier': 0, 'bonus_rate': 0.000},
                {'threshold_multiplier': 10, 'bonus_rate': 0.0020},
                {'threshold_multiplier': 25, 'bonus_rate': 0.0030},
                {'threshold_multiplier': 40, 'bonus_rate': 0.0040},
                {'threshold_multiplier': 60, 'bonus_rate': 0.0045},
            ],
            'secured_direct': [
                {'threshold_multiplier': 0, 'bonus_rate': 0.0010},
                {'threshold_multiplier': 10, 'bonus_rate': 0.0015},
                {'threshold_multiplier': 25, 'bonus_rate': 0.0025},
                {'threshold_multiplier': 40, 'bonus_rate': 0.0030},
            ],
            'unsecured': [
                {'threshold_multiplier': 0, 'bonus_rate': 0.000},
                {'threshold_multiplier': 4, 'bonus_rate': 0.004},
                {'threshold_multiplier': 12, 'bonus_rate': 0.008},
                {'threshold_multiplier': 16, 'bonus_rate': 0.01},
            ],
        }

        self.VINTAGE_SECURED_EFFORT_MATRIX_4_6M = {
            (0, 1): {(0, 0): 1200, (1, 1): 1500, (2, 2): 1800, (3, float('inf')): 2000},
            (2, 2): {(0, 0): 1500, (1, 1): 1800, (2, 2): 2000, (3, float('inf')): 2500},
            (3, 3): {(0, 0): 1800, (1, 1): 2000, (2, 2): 3000, (3, float('inf')): 3500},
            (4, float('inf')): {(0, 0): 2000, (1, 1): 2500, (2, 2): 3500, (3, float('inf')): 4000},
        }

        self.VINTAGE_UNSECURED_EFFORT_MATRIX_4_6M = {
            (0, 3): {(0, 0): 0, (1, 1): 200, (2, 2): 300, (3, 4): 400, (5, float('inf')): 500},
            (4, 5): {(0, 0): 200, (1, 1): 250, (2, 2): 350, (3, 4): 450, (5, float('inf')): 550},
            (6, 7): {(0, 0): 250, (1, 1): 350, (2, 2): 450, (3, 4): 550, (5, float('inf')): 650},
            (8, 9): {(0, 0): 350, (1, 1): 450, (2, 2): 550, (3, 4): 650, (5, float('inf')): 750},
            (10, float('inf')): {(0, 0): 450, (1, 1): 550, (2, 2): 650, (3, 4): 750, (5, float('inf')): 1000},
        }

        # Milestone Configuration
        self.MILESTONE_THRESHOLDS = {
            'silver': {'min_secured_files': 2, 'min_volume': 1500000},
            'gold': {'min_secured_files': 3, 'min_volume': 2000000},
            'platinum': {'min_secured_files': 4, 'min_volume': 3000000},
        }

        # Configurable evaluation periods based on financial quarters
        # Q1: Apr-Jun, Q2: Jul-Sep, Q3: Oct-Dec, Q4: Jan-Mar
        # Will be calculated dynamically based on joining date
        self.EVALUATION_PERIOD_MONTHS = None  # Will be set by calculate_evaluation_period()

        self.MILESTONE_QUALIFYING_MONTHS = {
            'silver': None,  # Will be calculated dynamically
            'gold': None,    # Will be calculated dynamically
            'platinum': None # Will be calculated dynamically
        }

        self.MILESTONE_SALARY_HIKE = {
            'silver': {'below_25k': 0.08, 'above_25k': 0.05},
            'gold': {'below_25k': 0.15, 'above_25k': 0.12},
            'platinum': {'below_25k': 0.20, 'above_25k': 0.15},
        }

        self.MILESTONE_SATURDAY_OFF = {
            'silver': 1,
            'gold': 2,
            'platinum': 3,
        }

    def calculate_evaluation_period(self, joining_date: datetime) -> int:
        """
        Calculate evaluation period based on financial quarter and joining date

        Financial Quarters:
        Q1: April 1st to June 30th
        Q2: July 1st to September 30th
        Q3: October 1st to December 31st
        Q4: January 1st to March 31st

        Logic:
        - If joins in 1st month of quarter: 7 months evaluation
        - If joins in 2nd month of quarter: 6 months evaluation (BAU)
        - If joins in 3rd month of quarter: 5 months evaluation

        Returns:
            int: Number of months for evaluation (5, 6, or 7)
        """

        # Determine which month of the financial quarter
        month = joining_date.month

        # Map calendar months to financial quarter months
        if month in [4, 7, 10, 1]:  # 1st month of quarter
            return 7
        elif month in [5, 8, 11, 2]:  # 2nd month of quarter
            return 6
        elif month in [6, 9, 12, 3]:  # 3rd month of quarter
            return 5
        else:
            return 6  # Default BAU

    def update_milestone_qualifying_months(self, evaluation_period: int):
        """Update milestone qualifying months based on evaluation period"""
        self.MILESTONE_QUALIFYING_MONTHS = {
            'silver': 3 if evaluation_period == 5 else (4 if evaluation_period == 6 else 4),
            'gold': 3 if evaluation_period == 5 else (4 if evaluation_period == 6 else 5),
            'platinum': 3 if evaluation_period == 5 else (4 if evaluation_period == 6 else 5),
        }

    def get_vintage_category(self, joining_date: datetime, evaluation_date: datetime) -> str:
        """Determine vintage category based on joining and evaluation dates"""
        months_diff = (evaluation_date.year - joining_date.year) * 12 + (evaluation_date.month - joining_date.month)

        if months_diff <= 3:
            return "0-3M"
        elif months_diff <= 6:
            return "4-6M"
        else:
            return "BAU"

    def is_month_evaluated(self, joining_date: datetime, evaluation_date: datetime) -> bool:
        """
        Check if a month should be evaluated based on joining date
        All months are evaluated, but joining month after 10th gets special treatment
        """
        return True

    def is_joining_month_after_10th(self, joining_date: datetime, evaluation_date: datetime) -> bool:
        """
        Check if this is the joining month and joined after 10th
        """
        if joining_date.year == evaluation_date.year and joining_date.month == evaluation_date.month:
            return joining_date.day > 10
        return False

    def find_matrix_value(self, matrix: Dict, disbursals: int, logins: int) -> int:
        """
        Find value from effort matrix based on disbursals and logins

        CORRECTED Matrix structure interpretation:
        - First level key (a, b): LOGIN range (not disbursal!)
        - Second level key (c, d): DISBURSAL range (not login!)
        - Value: payout amount

        The axes were swapped in previous implementation!
        Example: (3, inf): {(1, 1): 3500} means:
        - 3+ logins AND exactly 1 disbursal = ₹3500
        """
        # Find the LOGIN range first (outer key)
        login_key = None
        for key in matrix.keys():
            if key[0] <= logins <= key[1]:
                login_key = key
                break

        if login_key is None:
            return 0

        # Find the DISBURSAL range within the login range (inner key)
        disbursal_matrix = matrix[login_key]
        for key in disbursal_matrix.keys():
            if key[0] <= disbursals <= key[1]:
                return disbursal_matrix[key]

        return 0

    def calculate_slab_bonus(self, volume: float, salary: float, slabs: List[Dict]) -> Tuple[float, List[Dict]]:
        """
        Calculate bonus using slab system (like income tax)
        Returns total bonus and breakdown by slab
        """
        total_bonus = 0
        breakdown = []

        for i, slab in enumerate(slabs):
            current_threshold = slab['threshold_multiplier'] * salary

            if i == len(slabs) - 1:  # Last slab
                if volume > current_threshold:
                    taxable_amount = volume - current_threshold
                    slab_bonus = taxable_amount * slab['bonus_rate']
                    total_bonus += slab_bonus
                    breakdown.append({
                        'slab': f"Above {slab['threshold_multiplier']}x salary",
                        'threshold': current_threshold,
                        'rate': slab['bonus_rate'],
                        'taxable_amount': taxable_amount,
                        'bonus': slab_bonus
                    })
            else:
                next_threshold = slabs[i + 1]['threshold_multiplier'] * salary
                if volume > current_threshold:
                    taxable_amount = min(volume, next_threshold) - current_threshold
                    if taxable_amount > 0:
                        slab_bonus = taxable_amount * slab['bonus_rate']
                        total_bonus += slab_bonus
                        breakdown.append({
                            'slab': f"{slab['threshold_multiplier']}x to {slabs[i + 1]['threshold_multiplier']}x salary",
                            'threshold': current_threshold,
                            'rate': slab['bonus_rate'],
                            'taxable_amount': taxable_amount,
                            'bonus': slab_bonus
                        })

        return total_bonus, breakdown

    def calculate_monthly_bonus(self, rm_data: Dict) -> Dict:
        """
        Calculate complete monthly bonus for an RM

        Args:
            rm_data: Dictionary containing RM's monthly performance data

        Returns:
            Dictionary with detailed bonus breakdown
        """

        # Parse dates
        joining_date = datetime.strptime(rm_data['joining_date'], '%d/%m/%Y')
        evaluation_date = datetime.strptime(rm_data['evaluation_date'], '%d/%m/%Y')

        # Calculate evaluation period dynamically
        evaluation_period = self.calculate_evaluation_period(joining_date)
        self.EVALUATION_PERIOD_MONTHS = evaluation_period
        self.update_milestone_qualifying_months(evaluation_period)

        # Check for special case: joining month after 10th
        is_joining_after_10th = self.is_joining_month_after_10th(joining_date, evaluation_date)

        # Determine vintage category
        vintage_category = self.get_vintage_category(joining_date, evaluation_date)

        # Select appropriate slabs and matrices based on vintage
        if vintage_category == "0-3M":
            volume_slabs = self.VINTAGE_VOLUME_SLABS_0_3M
            secured_effort_matrix = self.VINTAGE_SECURED_EFFORT_MATRIX_0_3M
            unsecured_effort_matrix = self.VINTAGE_UNSECURED_EFFORT_MATRIX_0_3M
        elif vintage_category == "4-6M":
            volume_slabs = self.VINTAGE_VOLUME_SLABS_4_6M
            secured_effort_matrix = self.VINTAGE_SECURED_EFFORT_MATRIX_4_6M
            unsecured_effort_matrix = self.VINTAGE_UNSECURED_EFFORT_MATRIX_4_6M
        else:  # BAU
            volume_slabs = self.VOLUME_PAYOUT_SLABS
            secured_effort_matrix = self.BAU_SECURED_EFFORT_MATRIX
            unsecured_effort_matrix = self.BAU_UNSECURED_EFFORT_MATRIX

        # Calculate Volume Bonus
        secured_bonus, secured_breakdown = self.calculate_slab_bonus(
            rm_data['secured_volume'], rm_data['base_salary'], volume_slabs['secured']
        )

        secured_direct_bonus, secured_direct_breakdown = self.calculate_slab_bonus(
            rm_data['secured_direct_volume'], rm_data['base_salary'], volume_slabs['secured_direct']
        )

        unsecured_bonus, unsecured_breakdown = self.calculate_slab_bonus(
            rm_data['unsecured_volume'], rm_data['base_salary'], volume_slabs['unsecured']
        )

        total_volume_bonus = secured_bonus + secured_direct_bonus + unsecured_bonus

        # Calculate Effort Bonus (using corrected matrix access)
        secured_effort_bonus = self.find_matrix_value(
            secured_effort_matrix,
            rm_data['secured_disbursals'],
            rm_data['secured_logins']
        )

        unsecured_effort_bonus = self.find_matrix_value(
            unsecured_effort_matrix,
            rm_data['unsecured_disbursals'],
            rm_data['unsecured_logins']
        )

        total_effort_bonus = secured_effort_bonus + unsecured_effort_bonus

        # Total Monthly Bonus
        total_bonus = total_volume_bonus + total_effort_bonus

        # Special handling for joining after 10th of the month
        if is_joining_after_10th:
            # Calculate pro-rated ₹5000 based on days worked
            days_in_month = 30  # Standard assumption
            days_worked = days_in_month - joining_date.day + 1
            pro_rated_5000 = (5000 * days_worked) / days_in_month

            # Use higher of pro-rated ₹5000 or calculated payout
            total_bonus = max(pro_rated_5000, total_bonus)
            special_treatment = f"Joined after 10th: Used max(₹{pro_rated_5000:,.2f} pro-rated, ₹{total_volume_bonus + total_effort_bonus:,.2f} calculated)"
        else:
            special_treatment = "Normal calculation"

        return {
            'total_bonus': total_bonus,
            'evaluation_status': 'Evaluated',
            'vintage_category': vintage_category,
            'evaluation_period_months': evaluation_period,
            'volume_bonus': total_volume_bonus,
            'effort_bonus': total_effort_bonus,
            'special_treatment': special_treatment,
            'breakdown': {
                'volume_breakdown': {
                    'secured': {'bonus': secured_bonus, 'details': secured_breakdown},
                    'secured_direct': {'bonus': secured_direct_bonus, 'details': secured_direct_breakdown},
                    'unsecured': {'bonus': unsecured_bonus, 'details': unsecured_breakdown}
                },
                'effort_breakdown': {
                    'secured_effort': secured_effort_bonus,
                    'unsecured_effort': unsecured_effort_bonus
                }
            }
        }

    def calculate_milestone_bonus(self, rm_monthly_data: List[Dict]) -> Dict:
        """
        Calculate milestone bonus based on sustained performance

        Args:
            rm_monthly_data: List of monthly performance data for evaluation period

        Returns:
            Dictionary with milestone bonus details
        """

        if not rm_monthly_data:
            return {'milestone_achieved': None, 'rewards': {}}

        # Analyze performance across months
        qualifying_months = {'silver': 0, 'gold': 0, 'platinum': 0}

        for month_data in rm_monthly_data:
            secured_files = month_data.get('secured_disbursals', 0)
            total_volume = month_data.get('secured_volume', 0) + month_data.get('unsecured_volume', 0)

            # Check each milestone
            if (secured_files >= self.MILESTONE_THRESHOLDS['platinum']['min_secured_files'] and
                total_volume >= self.MILESTONE_THRESHOLDS['platinum']['min_volume']):
                qualifying_months['platinum'] += 1
            elif (secured_files >= self.MILESTONE_THRESHOLDS['gold']['min_secured_files'] and
                  total_volume >= self.MILESTONE_THRESHOLDS['gold']['min_volume']):
                qualifying_months['gold'] += 1
            elif (secured_files >= self.MILESTONE_THRESHOLDS['silver']['min_secured_files'] and
                  total_volume >= self.MILESTONE_THRESHOLDS['silver']['min_volume']):
                qualifying_months['silver'] += 1

        # Determine highest milestone achieved
        milestone_achieved = None
        base_salary = rm_monthly_data[0].get('base_salary', 0)

        if qualifying_months['platinum'] >= self.MILESTONE_QUALIFYING_MONTHS['platinum']:
            milestone_achieved = 'platinum'
        elif qualifying_months['gold'] >= self.MILESTONE_QUALIFYING_MONTHS['gold']:
            milestone_achieved = 'gold'
        elif qualifying_months['silver'] >= self.MILESTONE_QUALIFYING_MONTHS['silver']:
            milestone_achieved = 'silver'

        # Calculate rewards
        rewards = {}
        if milestone_achieved:
            salary_category = 'below_25k' if base_salary < 25000 else 'above_25k'
            salary_hike_percentage = self.MILESTONE_SALARY_HIKE[milestone_achieved][salary_category]
            salary_hike_amount = base_salary * salary_hike_percentage
            saturday_offs = self.MILESTONE_SATURDAY_OFF[milestone_achieved]

            rewards = {
                'milestone': milestone_achieved,
                'salary_hike_percentage': salary_hike_percentage * 100,
                'salary_hike_amount': salary_hike_amount,
                'saturday_offs_per_month': saturday_offs,
                'qualifying_months': qualifying_months[milestone_achieved],
                'required_months': self.MILESTONE_QUALIFYING_MONTHS[milestone_achieved]
            }

        return {
            'milestone_achieved': milestone_achieved,
            'rewards': rewards,
            'qualifying_months': qualifying_months
        }

In [34]:
import pandas as pd
from datetime import datetime
import os

def process_rm_bonus_csv(input_csv_path, output_csv_path=None):
    """
    Process RM bonus calculations from CSV input and generate CSV output

    Expected CSV columns:
    - rm_id, rm_name, joining_date, evaluation_date, base_salary
    - secured_logins, secured_disbursals, unsecured_logins, unsecured_disbursals
    - secured_volume, secured_direct_volume, unsecured_volume

    Args:
        input_csv_path (str): Path to input CSV file
        output_csv_path (str): Path for output CSV file (optional)

    Returns:
        DataFrame: Processed results
    """

    # Initialize calculator
    calculator = RMBonusCalculator()

    print("📁 Reading CSV file...")
    try:
        # Read input CSV
        df_input = pd.read_csv(input_csv_path)
        print(f"✅ Successfully loaded {len(df_input)} records")

        # Display sample of input data
        print(f"\n📊 Input Data Sample:")
        print(df_input.head())

        # Validate required columns
        required_columns = [
            'rm_id', 'rm_name', 'joining_date', 'evaluation_date', 'base_salary',
            'secured_logins', 'secured_disbursals', 'unsecured_logins', 'unsecured_disbursals',
            'secured_volume', 'secured_direct_volume', 'unsecured_volume'
        ]

        missing_columns = [col for col in required_columns if col not in df_input.columns]
        if missing_columns:
            print(f"❌ Missing required columns: {missing_columns}")
            return None

    except Exception as e:
        print(f"❌ Error reading CSV: {e}")
        return None

    print(f"\n🔄 Processing {len(df_input)} RM records...")

    # Process each record
    results = []
    errors = []

    for idx, row in df_input.iterrows():
        try:
            # Prepare data for calculator
            rm_data = {
                # Use the correct date format '%d/%m/%y'
                'joining_date': datetime.strptime(row['joining_date'], '%d/%m/%y').strftime('%d/%m/%Y'),
                'evaluation_date': datetime.strptime(row['evaluation_date'], '%d/%m/%y').strftime('%d/%m/%Y'),
                'base_salary': float(row['base_salary']),
                'secured_logins': int(row['secured_logins']),
                'secured_disbursals': int(row['secured_disbursals']),
                'unsecured_logins': int(row['unsecured_logins']),
                'unsecured_disbursals': int(row['unsecured_disbursals']),
                'secured_volume': float(row['secured_volume']),
                'secured_direct_volume': float(row['secured_direct_volume']),
                'unsecured_volume': float(row['unsecured_volume'])
            }

            # Calculate bonus
            result = calculator.calculate_monthly_bonus(rm_data)

            # Prepare output record
            output_record = {
                'rm_id': row['rm_id'],
                'rm_name': row['rm_name'],
                'joining_date': rm_data['joining_date'], # Use the parsed date
                'evaluation_date': rm_data['evaluation_date'], # Use the parsed date
                'base_salary': row['base_salary'],
                'vintage_category': result['vintage_category'],
                'evaluation_period_months': result.get('evaluation_period_months', 'N/A'),
                'evaluation_status': result['evaluation_status'],
                'secured_logins': row['secured_logins'],
                'secured_disbursals': row['secured_disbursals'],
                'unsecured_logins': row['unsecured_logins'],
                'unsecured_disbursals': row['unsecured_disbursals'],
                'secured_volume': row['secured_volume'],
                'secured_direct_volume': row['secured_direct_volume'],
                'unsecured_volume': row['unsecured_volume'],
                'volume_bonus': round(result['volume_bonus'], 2),
                'effort_bonus': round(result['effort_bonus'], 2),
                'secured_effort_bonus': round(result['breakdown']['effort_breakdown']['secured_effort'], 2),
                'unsecured_effort_bonus': round(result['breakdown']['effort_breakdown']['unsecured_effort'], 2),
                'secured_volume_bonus': round(result['breakdown']['volume_breakdown']['secured']['bonus'], 2),
                'secured_direct_bonus': round(result['breakdown']['volume_breakdown']['secured_direct']['bonus'], 2),
                'unsecured_volume_bonus': round(result['breakdown']['volume_breakdown']['unsecured']['bonus'], 2),
                'total_monthly_bonus': round(result['total_bonus'], 2)
            }

            results.append(output_record)

            # Progress indicator
            if (idx + 1) % 10 == 0:
                print(f"  ✅ Processed {idx + 1}/{len(df_input)} records")

        except Exception as e:
            error_record = {
                'rm_id': row.get('rm_id', f'Row_{idx}'),
                'error': str(e)
            }
            errors.append(error_record)
            print(f"  ❌ Error processing row {idx + 1}: {e}")

    # Create output DataFrame
    df_output = pd.DataFrame(results)

    print(f"\n📊 Processing Complete!")
    print(f"✅ Successfully processed: {len(results)} records")
    print(f"❌ Errors encountered: {len(errors)} records")

    if errors:
        print(f"\n⚠️ Error Details:")
        for error in errors:
            print(f"  • RM ID {error['rm_id']}: {error['error']}")

    # Generate output file path if not provided
    if output_csv_path is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_csv_path = f"rm_bonus_results_{timestamp}.csv"

    # Save to CSV
    try:
        df_output.to_csv(output_csv_path, index=False)
        print(f"\n💾 Results saved to: {output_csv_path}")

        # Display sample output
        print(f"\n📈 Output Sample:")
        print(df_output[['rm_id', 'rm_name', 'vintage_category', 'total_monthly_bonus', 'volume_bonus', 'effort_bonus']].head())

        # Summary statistics
        print(f"\n📊 Summary Statistics:")
        print(f"Average Total Bonus: ₹{df_output['total_monthly_bonus'].mean():,.2f}")
        print(f"Maximum Total Bonus: ₹{df_output['total_monthly_bonus'].max():,.2f}")
        print(f"Minimum Total Bonus: ₹{df_output['total_monthly_bonus'].min():,.2f}")
        print(f"Total Bonus Payout: ₹{df_output['total_monthly_bonus'].sum():,.2f}")

        # Vintage distribution
        print(f"\n📈 Vintage Distribution:")
        vintage_counts = df_output['vintage_category'].value_counts()
        for vintage, count in vintage_counts.items():
            print(f"  {vintage}: {count} RMs")

    except Exception as e:
        print(f"❌ Error saving CSV: {e}")
        return df_output

    return df_output

## one on one RM output

In [42]:
# Test the system with provided data
def test_bonus_calculator():
    """Test the bonus calculator with provided sample data"""

    calculator = RMBonusCalculator()

    # Test Case 1: Original data
    rm_data_1 = {
        'joining_date': '02/04/2025',
        'evaluation_date': '31/05/2025',
        'base_salary': 33750,
        'secured_logins': 2,
        'secured_disbursals': 3,
        'unsecured_logins': 0,
        'unsecured_disbursals': 0,
        'secured_volume': 1800000,
        'secured_direct_volume': 1800000,
        'unsecured_volume': 0
    }

    # Test Case 2: New case with matrix issue
    rm_data_2 = {
        'joining_date': '14/04/2025',
        'evaluation_date': '31/05/2025',
        'base_salary': 24833,
        'secured_logins': 2,
        'secured_disbursals': 2,
        'unsecured_logins': 0,
        'unsecured_disbursals': 0,
        'secured_volume': 630000,
        'secured_direct_volume': 630000,
        'unsecured_volume': 0
    }

    # Test Case 3: New case with matrix issue
    rm_data_3 = {
        'joining_date': '15/03/2025',
        'evaluation_date': '31/05/2025',
        'base_salary': 29167,
        'secured_logins': 2,
        'secured_disbursals': 4,
        'unsecured_logins': 0,
        'unsecured_disbursals': 0,
        'secured_volume': 1894362,
        'secured_direct_volume': 1894362,
        'unsecured_volume': 0
    }

    print("🏦 RM Bonus Calculator - UPDATED VERSION")
    print("=" * 60)

    # Test both cases
    for i, rm_data in enumerate([rm_data_1, rm_data_2, rm_data_3], 1):
        print(f"\n{'='*20} TEST CASE {i} {'='*20}")

        # Calculate monthly bonus
        result = calculator.calculate_monthly_bonus(rm_data)

        print(f"\n📊 RM Performance Summary:")
        print(f"Joining Date: {rm_data['joining_date']}")
        print(f"Evaluation Date: {rm_data['evaluation_date']}")
        print(f"Base Salary: ₹{rm_data['base_salary']:,}")
        print(f"Vintage Category: {result['vintage_category']}")
        print(f"Evaluation Period: {result.get('evaluation_period_months', 'N/A')} months")
        print(f"Evaluation Status: {result['evaluation_status']}")

        print(f"\n💰 Monthly Bonus Breakdown:")
        print(f"Volume Bonus: ₹{result['volume_bonus']:,.2f}")
        print(f"Effort Bonus: ₹{result['effort_bonus']:,.2f}")
        print(f"Total Monthly Bonus: ₹{result['total_bonus']:,.2f}")

                # Volume breakdown
        print(f"\n📈 Volume Bonus Details:")
        vol_breakdown = result['breakdown']['volume_breakdown']

        print(f"  Secured Volume Bonus: ₹{vol_breakdown['secured']['bonus']:,.2f}")
        for detail in vol_breakdown['secured']['details']:
            print(f"    • {detail['slab']}: ₹{detail['taxable_amount']:,.0f} @ {detail['rate']*100:.2f}% = ₹{detail['bonus']:,.2f}")

        print(f"  Secured Direct Bonus: ₹{vol_breakdown['secured_direct']['bonus']:,.2f}")
        for detail in vol_breakdown['secured_direct']['details']:
            print(f"    • {detail['slab']}: ₹{detail['taxable_amount']:,.0f} @ {detail['rate']*100:.2f}% = ₹{detail['bonus']:,.2f}")

        print(f"  Unsecured Volume Bonus: ₹{vol_breakdown['unsecured']['bonus']:,.2f}")

        # Effort breakdown with matrix explanation
        print(f"\n⚡ Effort Bonus Matrix Analysis:")
        effort_breakdown = result['breakdown']['effort_breakdown']
        print(f"  Secured Effort: {rm_data['secured_logins']} logins, {rm_data['secured_disbursals']} disbursals → ₹{effort_breakdown['secured_effort']:,.2f}")
        print(f"  Unsecured Effort: {rm_data['unsecured_logins']} logins, {rm_data['unsecured_disbursals']} disbursals → ₹{effort_breakdown['unsecured_effort']:,.2f}")


                # Effort breakdown with detailed explanation
        print(f"\n⚡ Effort Bonus Details:")
        effort_breakdown = result['breakdown']['effort_breakdown']
        print(f"  Secured Effort Bonus: ₹{effort_breakdown['secured_effort']:,.2f}")
        print(f"    (Based on {rm_data['secured_disbursals']} disbursals, {rm_data['secured_logins']} logins)")
        print(f"  Unsecured Effort Bonus: ₹{effort_breakdown['unsecured_effort']:,.2f}")
        print(f"    (Based on {rm_data['unsecured_disbursals']} disbursals, {rm_data['unsecured_logins']} logins)")
        print(f"    📝 Matrix Logic: 0 disbursals → use (0,0) entry regardless of logins = ₹0")

    return result

# Additional utility functions for comprehensive analysis
def create_performance_dashboard(monthly_data_list: List[Dict]) -> pd.DataFrame:
    """Create a performance dashboard from multiple months of data"""

    calculator = RMBonusCalculator()
    dashboard_data = []

    for month_data in monthly_data_list:
        result = calculator.calculate_monthly_bonus(month_data)

        dashboard_data.append({
            'Month': month_data['evaluation_date'],
            'Vintage': result['vintage_category'],
            'Volume_Bonus': result['volume_bonus'],
            'Effort_Bonus': result['effort_bonus'],
            'Total_Bonus': result['total_bonus'],
            'Secured_Volume': month_data['secured_volume'],
            'Unsecured_Volume': month_data['unsecured_volume'],
            'Secured_Disbursals': month_data['secured_disbursals'],
            'Secured_Logins': month_data['secured_logins']
        })

    return pd.DataFrame(dashboard_data)

def analyze_milestone_progression(rm_monthly_data: List[Dict]) -> Dict:
    """Analyze milestone progression over time"""

    calculator = RMBonusCalculator()
    milestone_result = calculator.calculate_milestone_bonus(rm_monthly_data)

    # Add month-by-month analysis
    monthly_milestone_analysis = []
    for month_data in rm_monthly_data:
        secured_files = month_data.get('secured_disbursals', 0)
        total_volume = month_data.get('secured_volume', 0) + month_data.get('unsecured_volume', 0)

        monthly_milestone_analysis.append({
            'month': month_data['evaluation_date'],
            'secured_files': secured_files,
            'total_volume': total_volume,
            'meets_silver': (secured_files >= 2 and total_volume >= 1500000),
            'meets_gold': (secured_files >= 3 and total_volume >= 2000000),
            'meets_platinum': (secured_files >= 4 and total_volume >= 3000000)
        })

    milestone_result['monthly_analysis'] = monthly_milestone_analysis
    return milestone_result

# Run the test
if __name__ == "__main__":
    test_result = test_bonus_calculator()

print("\n🚀 RM Bonus Calculator System Ready!")
print("Use the RMBonusCalculator class to calculate bonuses for any RM.")
print("Key methods:")
print("• calculate_monthly_bonus(rm_data) - Calculate monthly bonus")
print("• calculate_milestone_bonus(monthly_data_list) - Calculate milestone rewards")
print("• create_performance_dashboard(monthly_data_list) - Create performance dashboard")
print("• calculate_evaluation_period(joining_date) - Get evaluation period based on financial quarter")

🏦 RM Bonus Calculator - UPDATED VERSION

==================== TEST CASE 1 ====================

📊 RM Performance Summary:
Joining Date: 02/04/2025
Evaluation Date: 31/05/2025
Base Salary: ₹33,750
Vintage Category: 0-3M
Evaluation Period: 7 months
Evaluation Status: Evaluated

💰 Monthly Bonus Breakdown:
Volume Bonus: ₹9,961.88
Effort Bonus: ₹4,000.00
Total Monthly Bonus: ₹13,961.88

📈 Volume Bonus Details:
  Secured Volume Bonus: ₹6,131.25
    • 0x to 5x salary: ₹168,750 @ 0.00% = ₹0.00
    • 5x to 20x salary: ₹506,250 @ 0.25% = ₹1,265.62
    • 20x to 30x salary: ₹337,500 @ 0.35% = ₹1,181.25
    • 30x to 45x salary: ₹506,250 @ 0.45% = ₹2,278.12
    • Above 45x salary: ₹281,250 @ 0.50% = ₹1,406.25
  Secured Direct Bonus: ₹3,830.62
    • 0x to 10x salary: ₹337,500 @ 0.12% = ₹405.00
    • 10x to 25x salary: ₹506,250 @ 0.16% = ₹810.00
    • 25x to 40x salary: ₹506,250 @ 0.25% = ₹1,265.62
    • Above 40x salary: ₹450,000 @ 0.30% = ₹1,350.00
  Unsecured Volume Bonus: ₹0.00

⚡ Effort Bonus Mat

## File as output

In [35]:
process_rm_bonus_csv('/content/RM_Input.csv')

📁 Reading CSV file...
✅ Successfully loaded 150 records

📊 Input Data Sample:
    rm_id rm_name joining_date evaluation_date  base_salary  secured_logins  \
0  sf0023       s     28/02/25        31/05/25        21667             1.0   
1  sf0024       s     01/03/25        31/05/25        20500             1.0   
2  sf0029       s     04/03/25        31/05/25        30000             5.0   
3  sf0030       s     04/03/25        31/05/25        30000             4.0   
4  sf0033       s     10/03/25        31/05/25        30000             4.0   

   secured_disbursals  unsecured_logins  unsecured_disbursals  secured_volume  \
0                 1.0               2.0                   0.0        480643.0   
1                 0.0               0.0                   0.0             0.0   
2                 2.0               1.0                   0.0       1894698.0   
3                 1.0               0.0                   0.0       1797877.0   
4                 2.0               1.0   

,rm_id,rm_name,joining_date,evaluation_date,base_salary,vintage_category,evaluation_period_months,evaluation_status,secured_logins,secured_disbursals,...,secured_direct_volume,unsecured_volume,volume_bonus,effort_bonus,secured_effort_bonus,unsecured_effort_bonus,secured_volume_bonus,secured_direct_bonus,unsecured_volume_bonus,total_monthly_bonus
0,sf0023,s,28/02/2025,31/05/2025,21667,0-3M,6,Evaluated,1.0,1.0,...,480643.0,0.0,1660.43,2200,2000,200,978.07,682.36,0.0,3860.43
1,sf0024,s,01/03/2025,31/05/2025,20500,0-3M,5,Evaluated,1.0,0.0,...,0.0,0.0,0.00,1800,1800,0,0.00,0.00,0.0,1800.00
2,sf0029,s,04/03/2025,31/05/2025,30000,0-3M,5,Evaluated,5.0,2.0,...,1894698.0,0.0,11212.58,4000,4000,0,6923.49,4289.09,0.0,15212.58
3,sf0030,s,04/03/2025,31/05/2025,30000,0-3M,5,Evaluated,4.0,1.0,...,1797877.0,0.0,10438.02,3500,3500,0,6439.39,3998.63,0.0,13938.02
4,sf0033,s,10/03/2025,31/05/2025,30000,0-3M,5,Evaluated,4.0,2.0,...,1610592.0,200000.0,9459.74,4200,4000,200,5502.96,3436.78,520.0,13659.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,sf0208,s,07/05/2025,31/05/2025,20833,0-3M,6,Evaluated,5.0,1.0,...,412300.0,0.0,1346.69,3500,3500,0,770.34,576.35,0.0,4846.69
113,sf0211,s,07/05/2025,31/05/2025,32500,0-3M,6,Evaluated,3.0,0.0,...,0.0,0.0,0.00,3000,3000,0,0.00,0.00,0.0,3000.00
114,sf0213,s,08/05/2025,31/05/2025,29167,0-3M,6,Evaluated,2.0,0.0,...,0.0,0.0,0.00,2500,2500,0,0.00,0.00,0.0,2500.00
115,sf0214,s,09/05/2025,31/05/2025,21083,0-3M,6,Evaluated,2.0,0.0,...,0.0,0.0,0.00,2500,2500,0,0.00,0.00,0.0,2500.00
